In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import json
import pandas as pd
from pandas import DataFrame as df 
from requests.exceptions import MissingSchema

In [2]:
class crawly:
    def __init__(self, key, url):
        self.key = key
        self.url = url
        self.sub_pg = []
    
    def crawl(self):
        if self.key == '{:04d}'.format(0): #Dada Dallas
            self.sub_pg=[]
            site = requests.get(self.url)
            site.encoding = 'ISO-885901'
            soup = BeautifulSoup(site.text, 'html.parser')
            page_list = soup.find_all(class_='event-name headliners')
            for i in page_list:
                cont = i.contents[0]
                link = cont['href']
                self.sub_pg.append(link)
        elif self.key =='{:04d}'.format(1):
            self.sub_pg=[]
            site = requests.get(self.url)
            site.encoding = 'ISO-885901'
            soup = BeautifulSoup(site.text, 'html.parser')
            page_list = soup.find_all("div", class_="bottom_btns")
            for i in page_list:
                cont = i.contents[1]
                self.sub_pg.append(i.contents[1]['href'])
        else:
            print("Invalid Key")
            return[]
        return self.sub_pg

In [5]:
class creepy:
    prd = {  
        'Org Key':"",
        'Event Key':"",
        'Event Title' :"",
        'Topline': "",
        'Headliner':"",
        'Openers':"",
        'Date(s)':"",
        'Time(s)':"",
        'Price/Admission':'',
        'Age Restriction':'',
        'Event Description':'',
        'Venue':'',
        'Venue Info':'',
        'Street Name':'',
        'Address Line 2':',',
        'City':'',
        'State':'',
        'Postal Code':'',
        'Event Image URL':"",
        'Visit Event Website':'',
        'Venue Info':'',
        'Location Link':'',
        'Get Tickets':'',
        'URL':''}
    
    def __init__(self, key, pages):
        self.key = key
        self.pages = pages
        self.info = []
        self.count=0
        
    def creep(self):
        if self.key == '{:04d}'.format(0): #Dada Dallas
            for link in self.pages:
                event_info = creepy.prd.copy()
                on_sale = True
                event_info.update([('Org Key',self.key)])
                event_info.update([('Event Key','{:06d}'.format(self.count))])
                self.count+=1
                s_page = requests.get(link)
                #print(link) error finder
                sub_soup = BeautifulSoup(s_page.text, 'html.parser')
                sub_parse = sub_soup.find('meta', property ="og:title")
                event_info.update([('Event Title', sub_parse.attrs['content'])])
                sub_parse = sub_soup.find('section', class_="topline-info presented-by")
                if type(sub_parse) is type(None):
                    event_info.update([('Topline', "NA")])
                else:    
                    event_info.update([('Topline', sub_parse.get_text())])
                sub_parse = sub_soup.find('h1', class_="event-name headliners")
                event_info.update([('Headliner', sub_parse.get_text())])
                sub_parse = sub_soup.find('h2', class_="supports")
                if type(sub_parse) is type(None):
                    event_info.update([('Openers', 'NA')])
                else:
                    event_info.update([('Openers', sub_parse.get_text())])
                sub_parse = sub_soup.find('span', class_="dates")
                event_info.update([('Date(s)', sub_parse.get_text())])
                sub_parse = sub_soup.find('span', class_="start")
                event_info.update([('Time(s)', sub_parse.get_text())])
                sub_parse = sub_soup.find('span', class_="sales-ended inactive")
                if type(sub_parse) is type(None):
                    sub_parse = sub_soup.find('span', class_="price-range")
                    event_info.update([('Price/Admission', sub_parse.get_text())])
                else:
                    event_info.update([('Price/Admission', sub_parse.get_text())])
                    on_sale = False
                sub_parse = sub_soup.find('section', class_="age-restriction all-ages")
                if type(sub_parse) is type(None):
                    sub_parse = sub_soup.find('section', class_="age-restriction over-21")
                    event_info.update([('Age Restriction', sub_parse.get_text())])
                else:    
                    event_info.update([('Age Restriction', sub_parse.get_text())])
                sub_parse = sub_soup.find('article', class_="event-description")
                event_info.update([('Event Description', sub_parse.contents[1].text + " " +sub_parse.contents[3].text)])
                sub_parse = sub_soup.find('address', class_="venue-info")
                event_info.update([('Venue', sub_parse.contents[2])])
                sub_parse = sub_soup.find('meta', property="og:street-address")
                event_info.update([('Street Name', sub_parse.attrs['content'])])
                sub_parse = sub_soup.find('meta', property="og:locality")
                event_info.update([('City', sub_parse.attrs['content'])])
                sub_parse = sub_soup.find('meta', property="og:region")
                event_info.update([('State', sub_parse.attrs['content'])])
                sub_parse = sub_soup.find('meta', property="og:postal-code")
                event_info.update([('Postal Code', sub_parse.attrs['content'])])
                sub_parse = sub_soup.find('img')
                event_info.update([('Event Image URL', sub_parse.attrs['src'])])
                event_info.update([('URL',link)])
        
                if on_sale == True:
                    sub_parse = sub_soup.find('section', class_ = 'ticket-price')
                    for i in sub_parse.contents:
                        if type(i) == bs4.element.Tag and 'href' in i.attrs:
                            event_info.update([('Get Tickets', i.attrs['href'])])
                        elif type(i) == bs4.element.Tag:
                            for j in i.contents:
                                if type(j)== bs4.element.Tag and 'href' in j.attrs: 
                                    event_info.update([('Get Tickets', j.attrs['href'])])
                else:
                    event_info.update([('Get Tickets', event_info['Price/Admission'])])
            
            
                self.info.append(event_info) 
            return self.info
        elif self.key == '{:04d}'.format(1): #texas ballet
            for link in self.pages:
                event_info = self.prd.copy()
                event_info.update([('Org Key',self.key)])
                event_info.update([('Event Key','{:06d}'.format(self.count))])
                self.count+=1
                s_page = requests.get(link)
                sub_soup = BeautifulSoup(s_page.text, 'html.parser')
                sub_par = sub_soup.find('div', class_='title')
                #tvar = (sub_par.h1.text.replace('\n','') if sub_par.h1 else contents[0]).strip()
                vTitle = (sub_par.h1.text.replace('\n','') if sub_par.h1 else contents[0]).strip()
                event_info.update([("Event Title",vTitle)])
                
                
                
                self.info.append(event_info)
            return self.info
        else:
            print("Invalid Key")
            return[]    

In [20]:
class af_org:
    
    id_count=0
    orgs ={}
    org_data =pd.DataFrame()
    
    def __init__(self, name):
        self.name = name
        self.sUrl =""
        self.ssurl = ""
        self.ID = '{:04d}'.format(af_org.id_count)
        af_org.id_count+=1
        self.sub_pg = []
        self.data = []
        self.info= {"Org ID": self.ID,
                    "Org Name": self.name,
                    "Org URL": self.sUrl}
        af_org.orgs[self.ID] = self.name
        
    def set_sUrl(self, sUrl):
        try: 
            requests.get(sUrl) 
        except InvalidSchema as exception:
            print("URL is not complete: Please try again")
        except requests.ConnectionError as exception:
            print("URL does not exist on Internet. Please try Annother URL") 
        except MissingSchema as exception:
            print("URL is not complete: Please try again") 
        self.sUrl = sUrl
        self.info['Org URL'] = sUrl
        
    def update(self, data):
        self.data = data
        for i in range(len(af_org.org_data)):
            if af_org.org_data.at[i,"Org Key"]=='{:04d}'.format(int(self.ID)):
                af_org.org_data = af_org.org_data.drop(i)
        af_org.org_data = af_org.org_data.append(df.from_records(self.data), ignore_index = True)
        af_org.org_data = af_org.org_data.sort_values(by=['Org Key'])

In [21]:
dada = af_org("Dada Dallas")
dada.set_sUrl("https://www.dadadallas.com/calendar/")
dada.sub_pg = crawly(dada.ID, dada.sUrl).crawl()
dada.update(creepy(dada.ID, dada.sub_pg).creep())

In [22]:
tbt = af_org("Texas Ballet Theater")

In [30]:
tbt.set_sUrl('https://texasballettheater.org/20-21season/')
tbt.ssUrl = "https://texasballettheater.org/special-events/"
tbt.sub_pg = crawly(tbt.ID, tbt.sUrl).crawl()
tbt.update(creepy(tbt.ID, tbt.sub_pg).creep())


In [31]:
af_org.org_data

,Org Key,Event Key,Event Title,Topline,Headliner,Openers,Date(s),Time(s),Price/Admission,Age Restriction,...,Street Name,Address Line 2,City,State,Postal Code,Event Image URL,Visit Event Website,Location Link,Get Tickets,URL
0,0000,000000,"Vundabar - Tickets - Club Dada - Dallas, TX - ...",Margin Walker presents,Vundabar,NA,"Wed, December 2, 2020",7 PM,$15.00 - $18.00,All Ages,...,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,,,http://eventbrite.com/e/vundabar-new-date-club...,https://www.dadadallas.com/e/vundabar-10465726...
1,0000,000001,"Azizi Gibson - Tickets - Club Dada - Dallas, T...",Margin Walker presents,Azizi Gibson,NA,"Fri, December 4, 2020",7 PM,Sales Ended,All Ages,...,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,,,Sales Ended,https://www.dadadallas.com/e/azizi-gibson-9319...
2,0000,000002,SURFER BLOOD • Winter - Tickets - Club Dada - ...,presented by pofTX,Surfer Blood,Winter,"Wed, February 3, 2021",7 PM,$14.99,All Ages,...,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,,,https://www.eventbrite.com/e/surfer-blood-wint...,https://www.dadadallas.com/e/surfer-blood-wint...
3,0000,000003,NateWantsToBattle • Vespera - Tickets - Club D...,Margin Walker presents,NateWantsToBattle,Vespera,"Mon, March 29, 2021",7 PM,Sales Ended,All Ages,...,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,,,Sales Ended,https://www.dadadallas.com/e/natewantstobattle...
4,0000,000004,"Anti-Flag - Tickets - Club Dada - Dallas, TX -...",Margin Walker presents,Anti-Flag,NA,"Tue, May 18, 2021",7 PM,$18.00 - $20.00,All Ages,...,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,,,http://eventbrite.com/e/anti-flag-new-date-clu...,https://www.dadadallas.com/e/anti-flag-1050000...
5,0000,000005,Armor For Sleep - Tickets - Club Dada - Dallas...,Margin Walker presents,Armor For Sleep,NA,"Thu, July 15, 2021",7 PM,$25.00 - $28.00,All Ages,...,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,,,http://eventbrite.com/e/armor-for-sleep-club-d...,https://www.dadadallas.com/e/armor-for-sleep-9...
6,0001,000000,The Nutcracker,,,,,,,,...,,",",,,,,,,,
7,0001,000001,The Nutty Nutcracker,,,,,,,,...,,",",,,,,,,,
8,0001,000002,Serenade | Star Crossed | World Premiere by Ti...,,,,,,,,...,,",",,,,,,,,
9,0001,000003,Bartok | Image | Imbue,,,,,,,,...,,",",,,,,,,,
